[Reference](https://levelup.gitconnected.com/pathfinding-in-games-and-geospatial-applications-5e63ee18764b)

# Dijkstra

In [1]:
import heapq

def dijkstra(graph, start):
    dist = {node: float('inf') for node in graph}
    dist[start] = 0
    queue = [(0, start)]

    while queue:
        cost, current = heapq.heappop(queue)
        if cost > dist[current]:
            continue
        for neighbor, weight in graph[current]:
            new_cost = cost + weight
            if new_cost < dist[neighbor]:
                dist[neighbor] = new_cost
                heapq.heappush(queue, (new_cost, neighbor))

    return dist

# A*

In [2]:
import heapq

def astar(graph, start, goal, heuristic):
    open_set = [(0, start)]
    g = {start: 0}
    came_from = {}

    while open_set:
        _, current = heapq.heappop(open_set)
        if current == goal:
            return reconstruct_path(came_from, current)
        for neighbor, cost in graph[current]:
            tentative_g = g[current] + cost
            if tentative_g < g.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g[neighbor] = tentative_g
                f = tentative_g + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f, neighbor))

    return None

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

# BFS

In [3]:
from collections import deque

def bfs(graph, start, goal):
    queue = deque([start])
    visited = {start}
    came_from = {}

    while queue:
        current = queue.popleft()
        if current == goal:
            return reconstruct_path(came_from, current)
        for neighbor in graph[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                came_from[neighbor] = current
                queue.append(neighbor)

    return None

# DFS

In [4]:
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited)
    return visited

# Bidirectional Search

In [5]:
from collections import deque

def bidirectional_bfs(graph, start, goal):
    if start == goal:
        return [start]

    front_queue = deque([start])
    back_queue = deque([goal])

    front_parents = {start: None}
    back_parents = {goal: None}

    while front_queue and back_queue:
        meet_node = _expand_front(graph, front_queue, front_parents, back_parents)
        if meet_node:
            return _build_path(front_parents, back_parents, meet_node)

        meet_node = _expand_front(graph, back_queue, back_parents, front_parents)
        if meet_node:
            return _build_path(front_parents, back_parents, meet_node)

    return None  # No path found

def _expand_front(graph, queue, this_parents, other_parents):
    current = queue.popleft()
    for neighbor in graph.get(current, []):
        if neighbor not in this_parents:
            this_parents[neighbor] = current
            queue.append(neighbor)
            if neighbor in other_parents:
                return neighbor
    return None

def _build_path(front_parents, back_parents, meeting_point):
    path = []
    # build path from start to meeting_point
    node = meeting_point
    while node is not None:
        path.append(node)
        node = front_parents[node]
    path.reverse()
    # append path from meeting_point to goal
    node = back_parents[meeting_point]
    while node is not None:
        path.append(node)
        node = back_parents[node]
    return path

# Jump Point Search

In [6]:
import heapq

def jump_point_search(start, goal, grid):
    open_list = []
    heapq.heappush(open_list, (0, start))  # (f_score, node)

    came_from = {}
    g_score = {start: 0}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            return reconstruct_path(came_from, current)

        # prune_directions() is an application-specific function which
        # determines allowed directions to search from the current node
        for dx, dy in prune_directions(current, came_from):
            # jump() returns the next jump point in the given direction, or None
            jump_point = jump(grid, current[0], current[1], dx, dy, goal)
            if jump_point:
                tentative_g = g_score[current] + distance(current, jump_point)
                if jump_point not in g_score or tentative_g < g_score[jump_point]:
                    g_score[jump_point] = tentative_g
                    f_score = tentative_g + heuristic(jump_point, goal)
                    heapq.heappush(open_list, (f_score, jump_point))
                    came_from[jump_point] = current

    return None  # No path found

In [7]:
def jump(grid, x, y, dx, dy, goal):
    nx, ny = x + dx, y + dy
    if not is_walkable(grid, nx, ny):
        return None
    if (nx, ny) == goal:
        return (nx, ny)

    # check for forced neighbors: positions that require a turn due to obstacles
    if has_forced_neighbors(grid, nx, ny, dx, dy):
        return (nx, ny)

    # diagonal movement requires recursive checks along both axes
    if dx != 0 and dy != 0:
        if jump(grid, nx, ny, dx, 0, goal) or jump(grid, nx, ny, 0, dy, goal):
            return (nx, ny)

    return jump(grid, nx, ny, dx, dy, goal)

# Theta* and Any-Angle Pathfinding

In [8]:
def theta_star(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {start: start}
    g_score = {start: 0}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in neighbors(grid, current):
            parent = came_from[current]
            if line_of_sight(grid, parent, neighbor):
                tentative_g = g_score[parent] + distance(parent, neighbor)
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = parent
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score, neighbor))
            else:
                tentative_g = g_score[current] + distance(current, neighbor)
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score, neighbor))

    return None

# Hierarchical Pathfinding A*

In [9]:
def hierarchical_pathfinding(start, goal, grid, cluster_size):
    clusters = partition_grid(grid, cluster_size)
    entrances = identify_entrances(clusters)
    abstract_graph = build_abstract_graph(clusters, entrances)

    start_entrance = nearest_entrance(start, entrances)
    goal_entrance = nearest_entrance(goal, entrances)

    abstract_path = a_star_abstract(abstract_graph, start_entrance, goal_entrance)

    full_path = []
    for a, b in pairwise(abstract_path):
        cluster_path = a_star(grid, a, b)  # low-level A* in each cluster
        full_path.extend(cluster_path[:-1])  # avoid duplicate nodes

    full_path.append(goal)  # ensure goal is included
    return full_path

# Customizable Route Planning and Multi-Level Dijkstra

In [10]:
# Phase 1: Metric-Independent Preprocessing (run once)
def preprocess_graph(graph):
    # Recursively partition the graph into cells at multiple levels
    levels = []
    current_level = graph
    while size(current_level) > threshold:
        cells = partition(current_level)  # e.g., via METIS or KaHIP
        boundary_nodes = identify_boundary_nodes(cells)
        levels.append((cells, boundary_nodes))
        current_level = build_abstract_graph(cells, boundary_nodes)
    return levels

# Phase 2: Metric-Dependent Customization (repeatable for new weights)
def customize_weights(levels, edge_weights):
    for cells, boundary_nodes in levels:
        for cell in cells:
            # For each pair of boundary nodes in cell, compute shortest path shortcuts
            shortcuts = compute_shortcuts(cell, boundary_nodes[cell], edge_weights)
            update_cell_with_shortcuts(cell, shortcuts)

# Phase 3: Query (per path request)
def query_path(levels, start, goal, edge_weights):
    # Initialize search structures on all levels
    open_sets = init_open_sets(levels)
    distances = init_distances(levels, start, goal)

    while not all_open_sets_empty(open_sets):
        # Perform a level-aware multi-directional Dijkstra step
        for level in reversed(levels):
            relax_edges(open_sets[level], distances, edge_weights, level)
        # Check for meeting point between forward and backward searches

    return reconstruct_path(distances, start, goal)